Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as pyplot

Reading the csv file to generate a dataframe(df) and validating the df

In [2]:
dataframe = pd.read_csv("TIC_Data.csv", delimiter='\t',index_col=0)
dataframe.head(10)

,Old Scheme (in thousands),New Scheme (in thousands)
SALESPERSON,,
1,57,62
2,103,122
3,59,54
4,75,82
5,84,84
6,73,86
7,35,32
8,110,104
9,44,38


In [3]:
#Validating data
if pd.isnull(dataframe) is True:
    print("There is/are null data in the dataframe")
elif pd.isna(dataframe) is True:
    print("There is/are NA present in the dataframe")
#checking the dimentions of the data
dataframe.shape

(30, 2)

In [4]:
dataframe.describe()

,Old Scheme (in thousands),New Scheme (in thousands)
count,30.000000,30.000000
mean,68.033333,72.033333
std,20.455980,24.062395
min,28.000000,32.000000
25%,54.000000,55.000000
50%,67.000000,74.000000
75%,81.500000,85.750000
max,110.000000,122.000000


### 1. Finding the mean of the sales per salesman of od and new schemes. 

In [5]:
meanOS = dataframe["Old Scheme (in thousands)"].mean()
print("Old Scheme mean:  "+str(meanOS))

meanNS = dataframe["New Scheme (in thousands)"].mean()
print("New Scheme mean:  "+str(meanNS))

def outlierlims(dataframe):
    q1 = dataframe.quantile(0.25)
    q3 = dataframe.quantile(0.75)
    IQR = q3-q1
    mean = dataframe.mean()
    upperlimit = mean + 1.5*IQR
    lowerlimit = mean - 1.5*IQR
    return upperlimit,lowerlimit

upperOS,lowerOS = outlierlims(dataframe["Old Scheme (in thousands)"])
print(lowerOS,upperOS)
upperNS,lowerNS = outlierlims(dataframe["New Scheme (in thousands)"])
print(lowerNS,upperNS)

Old Scheme mean:  68.03333333333333
New Scheme mean:  72.03333333333333
26.78333333333333 109.28333333333333
25.90833333333333 118.15833333333333


### 2. Hypothesis Testing

Now, considering the problem statement

Null Hypothesis: There is no significant change in the sales of new schemes and old schemes.

Alternate Hypothesis: There is a significance change in the sales of the new schemes.

Note: 

1. We are considering a confidence coefficient (1-alpha) of 0.95.(Since the problem states a 5% significance test i.e alpha = 0.05) 

2. Used the function 'ttest_rel' because the two samples are dependent as they are taken from same population.

In [6]:
from scipy.stats import ttest_rel,ttest_1samp

t_statistic,p_value = ttest_rel(dataframe["Old Scheme (in thousands)"],dataframe["New Scheme (in thousands)"])

print("p_value: "+str(p_value))

p_value: 0.13057553961337662


### 3. Conclusion
Since the p_value is greater than the given value of alpha, we need to consider the null hypothesis. i.e we can conclude that the is no significance change between the sales of the new and old schemes. 

### 4. Expected mean and p_value calculation
As per the problem statement, for Titan to break even, the mean should increase by 5000 when compared to old scheme.

i.e old scheme mean = 68.033(in thousands)

    expected population mean = 68.033+5 (in thousands)
                             
                             = 73.033
                             
Now,

Null Hypothesis: Titan did not break even when compared to old scheme.

Alternate Hypothesis: Titan broke even i.e, there is 5000 increase in average output of each salesman


In [7]:
t_statistic,p_value = ttest_1samp(dataframe["New Scheme (in thousands)"],73.033)
print(t_statistic,p_value)

-0.22755007742596145 0.8215927010661443


Checking the p_value of the hypothesis test that there is a difference of 5000.

Null Hypothesis: difference between the old mean and the new mean is equal to 5000.

Alternate Hypothesis: difference is not equal to 5000.

In [8]:
t_statistic,p_value = ttest_1samp(dataframe["New Scheme (in thousands)"]-dataframe["Old Scheme (in thousands)"],5)
print(t_statistic,p_value)

-0.3889785955886094 0.7001334912613286


From the above we can conclude that there is a 70.013% chance that the difference is 5000.

In [9]:
# Calculating Power of Test
# Compute the difference in Means between 2 sample means and divide by pooled Standard Deviation 
# number of Observations/tuples
# Set the alpha value to 0.05 and alternative values 'two-sided' , 'larger' , 'smaller'
(np.mean(dataframe["New Scheme (in thousands)"]) - np.mean(dataframe["Old Scheme (in thousands)"])) / np.sqrt(((30-1)*np.var(dataframe["Old Scheme (in thousands)"])+(30-1)*np.var(dataframe["New Scheme (in thousands)"])) / (30+30-2))

0.18217632044690082

In [13]:
from statsmodels.stats.power import ttest_power
print(ttest_power(0.18, nobs=30, alpha=0.05, alternative='two-sided'))

0.15888754913103434
